In [27]:
import collections 
import re 
from d2l import torch as d2l

读取数据集

In [28]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    """
    将时间机器数据集加载到文本行的列表中

    Return
    -------
        文本行列表, 每个元素是一句话, 这句话中非英文字符替换为空格, 所有字符转换为小写
    """
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # re.sub('[^A-Za-z]+', ' ', line)将非英文字符替换为空格
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]  # 全部转换为小写

lines = read_time_machine()
print(f"文本总行数: {len(lines)}")
print(lines[0])
print(lines[10])

文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


词元化

In [29]:
def tokenize(lines, token='word'):
    """
    将文本行拆分为单词或字符词元
    
    Parameters
    -----------
        lines : 文本行列表, 每个元素是一句话
        token : 'word' 文本行拆分为单词 'char' 文本行拆分为字符
    """
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines] # list() 将字符串转换为字符列表
    else:
        print('错误: 未知词元类型: ' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


词表

In [30]:
class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """ 
        Parameters
        -----------
            tokens : 文本列表， 已拆解处理为单词
            min_freq : 出现次数<min_freq的单词, 直接移除(降低复杂度)
            reserved_tokens : 保留词元(特殊用途), 例如填充词元'<pad>', 序列开始词元'<bos>', 序列结束词元'<eos>'
        """
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        # items() (key, value)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True) # 所有单词，按出现频率降序排列
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens # 词元列表，先加入未知词元+保留词元
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)} # 词元 : 编号
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break 
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1   # 编号越大，出现频率越低 
        
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        """从Vocab类中取出词元对应的编号"""
        if not isinstance(tokens, (list, tuple)): # 当输入是单个词元时
            return self.token_to_idx.get(tokens, self.unk)   # 返回词元对应的编号
        return [self.__getitem__(token) for token in tokens] # 递归调用
    
    def to_tokens(self, indices):
        """编号转换为词元"""
        if not isinstance(indices, (list, tuple)):             # 单个编号
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices] # 多个编号
    
    @property  # 将类方法转换为类属性，可以用self.unk调用该方法
    def unk(self):  # 未知词元的索引为0
        return 0
    
    @property
    def token_freqs(self):
        return self._token_freqs
    
def count_corpus(tokens):
    """
    统计词元出现的频率

    Parameter
    ----------
        tokens : 文本列表, 已拆解处理为单词
    
    Return
    -------
        collections.Counter(tokens) 键为元素，值为对应元素的计数
    """
    # 这里的tokens是一维列表或二维列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    # collections.Counter 容器类，用于高效统计可哈希对象的出现次数
    # 键为元素，值为对应元素的计数
    return collections.Counter(tokens)

In [31]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [32]:
# 将每一个文本行转换成一个数字索引列表
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


整合所有功能

In [33]:
def load_corpus_time_machine(max_tokens=-1):
    """
    返回时光机器数据集的词元索引列表和词表, 此处使用字符实现文本词元化
    
    Parameter
    -----------
        max_tokens : 返回前max_tokens个词元索引
    """
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab 

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab) # vocab中包括unk, 空格，26个字符

(170580, 28)